In [1]:
from bs4 import BeautifulSoup
import requests
import sqlite3
import pandas as pd
import re

Импортируем необходимые библиотеки

In [2]:
url = "https://www.kp.ru/russia/moskva/restorany/"
page = requests.get(url)    
soup = BeautifulSoup(page.text, 'html.parser')

Указываем ссылку для парсинга и делаем на нее запрос

In [3]:
name = soup.select("div.text-block > h2.wp-block-heading")
info = soup.findAll('figure', class_='wp-block-table')
description = []
for i in soup.find_all('figcaption'):
    if i.figcaption:
        _  = i.figcaption.extract()
    description.append(i)

Парсим данные и распределяем их по соответствующим массивам

In [4]:
newName = []
for i in range(len(name)):
    newName.append(name[i].text)
newInfo = []
for i in range(len(info)):
    newInfo.append(info[i].text)
newDescription = []
for i in range(len(description)):
    newDescription.append(description[i].text)

Подготовка столбцов для удобной записи в дата фрейм в дальнейшем

In [5]:
df = pd.DataFrame()
df['Name'] = newName
df['Info'] = pd.Series(newInfo)
df['Description'] = pd.Series(newDescription)
df

,Name,Info,Description
0,«Dr. Живаго»,"АдресУлица Моховая, 15/1Средний чек2500 руб.",Акцент в ресторане сделан на русскую кухню.Фот...
1,«Северяне»,"АдресУлица Большая Никитская, 12 стр. 1Средний...",В ресторане подают блюда авторской интерпретац...
2,«Воронеж»,"АдресУлица Пречистенка, 4Средний чек2500 руб.",В этот ресторан обязательно нужно предваритель...
3,Selfie,"АдресНовинский бульвар, 31Средний чек3500 руб.",На работу поваров можно смотреть прямо из зала...
4,AQ Kitchen,"АдресУлица Большая грузинская, 69Средний чек20...",Рекомендуется брать гастрономический сет из пя...
5,«Пушкинъ»,"АдресТверской бульвар, 26аСредний чек3000 руб.","В ресторане имеется несколько залов, выполненн..."
6,«Стейк Хаус Бутчер»,"АдресСадовническая набережная, 75 и еще 5 фили...",Здесь предложат 13 видов стейков.Фото: pexels.com
7,«Консерватория»,"АдресУлица Неглинная, 4Средний чек3500 руб.",У ресторана есть летняя терраса с потрясающим ...
8,«Белый журавль»,"АдресФрунзенская набережная, 14/1Средний чек20...",Ресторан специализируется на корейской кухне.Ф...
9,Osteria Mario,"АдресМанежная площадь, 1 стр. 2 и еще 17 филиа...",В основе меню ресторана – итальянская кухня.Фо...


Получаем наш датафрейм. В столбце с описанием ресторанов имеется лишняя информация о фото, которое мы не можем просмотреть, а также последняя строка является лишней в виду того, что она не описывает никакого ресторана. Удалим ненужную информацию.

In [6]:
df['Description'] = df['Description'].str.split("Фото").str[0]
df = df.dropna()
df

,Name,Info,Description
0,«Dr. Живаго»,"АдресУлица Моховая, 15/1Средний чек2500 руб.",Акцент в ресторане сделан на русскую кухню.
1,«Северяне»,"АдресУлица Большая Никитская, 12 стр. 1Средний...",В ресторане подают блюда авторской интерпретации.
2,«Воронеж»,"АдресУлица Пречистенка, 4Средний чек2500 руб.",В этот ресторан обязательно нужно предваритель...
3,Selfie,"АдресНовинский бульвар, 31Средний чек3500 руб.",На работу поваров можно смотреть прямо из зала.
4,AQ Kitchen,"АдресУлица Большая грузинская, 69Средний чек20...",Рекомендуется брать гастрономический сет из пя...
5,«Пушкинъ»,"АдресТверской бульвар, 26аСредний чек3000 руб.","В ресторане имеется несколько залов, выполненн..."
6,«Стейк Хаус Бутчер»,"АдресСадовническая набережная, 75 и еще 5 фили...",Здесь предложат 13 видов стейков.
7,«Консерватория»,"АдресУлица Неглинная, 4Средний чек3500 руб.",У ресторана есть летняя терраса с потрясающим ...
8,«Белый журавль»,"АдресФрунзенская набережная, 14/1Средний чек20...",Ресторан специализируется на корейской кухне.
9,Osteria Mario,"АдресМанежная площадь, 1 стр. 2 и еще 17 филиа...",В основе меню ресторана – итальянская кухня.


In [9]:
connection = sqlite3.connect('articles.db')
df2 = pd.read_sql("SELECT * FROM articles", connection)
df2

,Name,Info,Description
0,«Dr. Живаго»,"АдресУлица Моховая, 15/1Средний чек2500 руб.",Акцент в ресторане сделан на русскую кухню.
1,«Северяне»,"АдресУлица Большая Никитская, 12 стр. 1Средний...",В ресторане подают блюда авторской интерпретации.
2,«Воронеж»,"АдресУлица Пречистенка, 4Средний чек2500 руб.",В этот ресторан обязательно нужно предваритель...
3,Selfie,"АдресНовинский бульвар, 31Средний чек3500 руб.",На работу поваров можно смотреть прямо из зала.
4,AQ Kitchen,"АдресУлица Большая грузинская, 69Средний чек20...",Рекомендуется брать гастрономический сет из пя...
5,«Пушкинъ»,"АдресТверской бульвар, 26аСредний чек3000 руб.","В ресторане имеется несколько залов, выполненн..."
6,«Стейк Хаус Бутчер»,"АдресСадовническая набережная, 75 и еще 5 фили...",Здесь предложат 13 видов стейков.
7,«Консерватория»,"АдресУлица Неглинная, 4Средний чек3500 руб.",У ресторана есть летняя терраса с потрясающим ...
8,«Белый журавль»,"АдресФрунзенская набережная, 14/1Средний чек20...",Ресторан специализируется на корейской кухне.
9,Osteria Mario,"АдресМанежная площадь, 1 стр. 2 и еще 17 филиа...",В основе меню ресторана – итальянская кухня.


Создаем подключение к SQLite 3, выгружаем в нее данные, собираем их в новый датафрейм и выводим его для проверки работы нашего кода. 